## Mount Drive

In [0]:
from google.colab import drive
drive.mount('/content/drive')

## Setting & Import

In [0]:
!pip3 install h5py

In [0]:
#Classifier
import os
import ast
import cv2 
import csv
import numpy as np 
import pandas as pd
from glob import glob
from tqdm import tqdm #used for progress bar
from dask import bag  #a tool for accelerating data processing
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.style as style
from PIL import Image, ImageDraw  #PIL:python imaging library
from tensorflow import keras
from keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from tensorflow.keras.metrics import top_k_categorical_accuracy
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

## Definition

In [0]:
class_paths = glob('/content/drive/My Drive/input/train/*.csv')
model_path = "./drive/My Drive/output/model/classifier/classifier_W.h5"
final_csv_dir = "./drive/My Drive/Final/output/csv/"
final_image_dir = "./drive/My Drive/Final/output/image/"
csv_dir = "./drive/My Drive/output/csv/"
image_dir = "./drive/My Drive/output/image/"

num_classes = 30
imheight, imwidth = 64, 64
ims_per_class = 13000

numstonames = {i: v[36:-4].replace(" ", "_") for i, v in enumerate(class_paths)}
print(numstonames)
namestonums = {v:k for k,v in numstonames.items()}

def draw_it(strokes):
  image = Image.new("P", (256,256), color=255)
  image_draw = ImageDraw.Draw(image)
  for stroke in ast.literal_eval(strokes):
    for i in range(len(stroke[0])-1): #number of positions , since # of x_coordinates = # of y_coordinates , just loop all x coordinates 
      image_draw.line([stroke[0][i],  #numeric values like [x, y, x, y, ...] 
                       stroke[1][i],
                       stroke[0][i+1], 
                       stroke[1][i+1]],
                       fill=0, width=5) #Color to use for the point
  image = image.resize((imheight, imwidth))
  return np.array(image)/255.

def top_3_accuracy(x,y): 
    t3 = top_k_categorical_accuracy(x,y, 3)
    return t3

def csvWriter(file_name, nparray):
  example = nparray.tolist()
  with open(file_name+'.csv', 'w', newline='') as csvfile:
     writer = csv.writer(csvfile, delimiter=',')
     writer.writerows(example)

def save_images(grapharray, save_index):
  grapharray = np.reshape(grapharray, (64, 64))
  plt.figure(figsize=(16, 16))
  plt.imshow(grapharray, interpolation='nearest', cmap='gray_r')
  plt.axis('off')
  plt.savefig(image_dir + image_label + "/" + image_label + "_{:04d}.png".format(save_index) )
  csvWriter(csv_dir + image_label + "/" + image_label + "_{:04d}".format(save_index), grapharray)


#Data Prepocessing

In [0]:
train_grand = []

for i,path in enumerate(tqdm(class_paths[0: num_classes])):
    train = pd.read_csv(path, nrows=ims_per_class)  #nrows:number of rows that needs to read , returns a dataframe
    imagebag = bag.from_sequence(train.drawing.values).map(draw_it) #index[0][i]:x-axis;index[1][i]:y-axis , they are passed arg 
    trainarray = np.array(imagebag.compute()) #convert a Dask bag to a list or Python iterable by calling compute()
    trainarray = np.reshape(trainarray, (ims_per_class, -1))   
    labelarray = np.full((train.shape[0], 1), i)  #train.shape[0] is 2000
    trainarray = np.concatenate((labelarray, trainarray), axis=1)
    train_grand.append(trainarray)
    
#less memory than np.concatenate
train_grand = np.array([train_grand.pop() for i in np.arange(num_classes)]) #pop for each classes
#reshape to 64*64 to put in CNN
train_grand = train_grand.reshape((-1, (imheight*imwidth+1)))

del trainarray
del train


## Validation Data Prepocessing

In [0]:
np.random.shuffle(train_grand)
y_train, X_train = train_grand[:, 0], train_grand[:, 1:]

valid = pd.read_csv('./drive/My Drive/input/valid.csv',nrows = ims_per_class)
imagebag = bag.from_sequence(valid.drawing.values).map(draw_it)
validarray = np.array(imagebag.compute())
validarray = np.reshape(validarray, (validarray.shape[0], imheight, imwidth, 1))

X_val = validarray
y_val = [namestonums[w.replace(" ", "_") ] for w in valid.word]

# del train_grand
y_train = keras.utils.to_categorical(y_train, num_classes)
X_train = X_train.reshape(X_train.shape[0], imheight, imwidth, 1)
y_val = keras.utils.to_categorical(y_val, num_classes)
X_val = X_val.reshape(X_val.shape[0], imheight, imwidth, 1)

## Classifier Model

In [0]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu', input_shape=(imheight, imwidth, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, kernel_size=(3, 3), padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(680, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

## Train Classifier

In [0]:
import time
starttime = time.time()

reduceLROnPlat = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, mode='auto', min_delta=0.005, cooldown=5, min_lr=0.0001)
earlystop = EarlyStopping(monitor='val_top_3_accuracy', mode='max', patience=5) 
#callbacks = [reduceLROnPlat, earlystop]
callbacks = [reduceLROnPlat]
#0.002->0.001
adam = keras.optimizers.Adam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy',optimizer=adam, metrics=['accuracy', top_3_accuracy])
model.fit(x=X_train, y=y_train, batch_size = 64, epochs = 15, validation_data = (X_val, y_val), shuffle=True, callbacks = callbacks, verbose = 1)

model.save("./drive/My Drive/output/model/classifier.h5")

endtime = time.time()

## Test Classification Result

In [0]:
test = pd.read_csv('./drive/My Drive/input/demo.csv')[['drawing']]
imagebag = bag.from_sequence(test.drawing.values).map(draw_it)
testarray = np.array(imagebag.compute())
testarray = np.reshape(testarray, (testarray.shape[0], imheight, imwidth, 1))
# model = load_model("./drive/My Drive/output/model/classifier.h5")
testpreds = model.predict(testarray, verbose=0)
ttvs = np.argsort(-testpreds)
for idx in ttvs[:,[0,1,2]]:
    print(numstonames[idx[0]])
    # print(numstonames[idx[1]])
    # print(numstonames[idx[2]])
    print('----------')

In [0]:
graph = pd.read_csv('./drive/My Drive/AI/input/output.csv',header=None)
grapharray = np.array(graph)
grapharray = np.where(grapharray[:] > 0.2,1,0)
grapharray = np.reshape(grapharray, (1,imheight, imwidth, 1))
graphpreds = model.predict(grapharray, verbose=0)
gvs = np.argsort(-graphpreds)
for idx in gvs[:]:
    for i in idx:
        print(numstonames[i])

## Testing Case


## Check Image

In [0]:
test_raw = pd.read_csv('./drive/My Drive/input/test.csv')
first_ids = test_raw.iloc[:].index
images = [ast.literal_eval(lst) for lst in test_raw.loc[first_ids, 'drawing'].values]
for index, drawing in enumerate(images, 0):
    
    plt.figure(figsize=(6,3))
    for x,y in drawing:
        plt.subplot(1,2,1)
        plt.plot(x, y, marker='.')
        plt.axis('on')


    plt.gca().invert_yaxis()
    plt.axis('equal')
    plt.show()  

In [0]:
#test
test = pd.read_csv('./drive/My Drive/input/test.csv')[['drawing']]
imagebag = bag.from_sequence(test.drawing.values).map(draw_it)
testarray = np.array(imagebag.compute())
testarray = np.reshape(testarray, (testarray.shape[0], imheight, imwidth, 1))
# model = load_model("./drive/My Drive/output/model/classifier.h5")
testpreds = model.predict(testarray, verbose=0)
ttvs = np.argsort(-testpreds)

for idx in ttvs[:,[0,1,2]]:
    print(numstonames[idx[0]])
    # print(numstonames[idx[1]])
    # print(numstonames[idx[2]])
    # print('----------')

predict_label = [numstonames[ttvs[i][0]] for i in range(ttvs.shape[0])]
test['word'] = predict_label
test.to_csv('./drive/My Drive/output/output.csv')


In [0]:
test_ans = pd.read_csv('./drive/My Drive/input/test_ans.csv',index_col=0)
# print(test_ans['word'])

In [0]:
# Compare
correct_count =0
test_count = 0
for idx in range(41):
  if(test_ans['word'][idx] == test['word'][idx].replace("_"," ")):correct_count+=1
  else: print(test_ans['word'][idx] + " " + test['word'][idx])
  test_count = test_count + 1

print(correct_count)
print(correct_count/test_count);

## Load Classifier Model


In [0]:
model = keras.models.load_model(model_path, custom_objects={'top_3_accuracy': top_3_accuracy})

## Check csv & image


In [0]:
image_label = "key"
image_path = image_dir + image_label + "/" + image_label+"_0002.png"
im = Image.open(image_path)
image_csv = pd.read_csv(csv_dir + image_label + "/" + image_label + "_0001.csv", skiprows=0, header=None, engine='python')
# print(image_csv)
image_csv = np.array(image_csv)
# print(image_csv.shape)

im = im.resize((64,64), Image.BILINEAR )
plt.imshow(im, interpolation='nearest', cmap='gray_r')

grapharray = image_csv
# grapharray = np.array(gray_image)
# for j in range(64):
#   for k in range(64):
#     if grapharray[j][k] < 0.2 : grapharray[j][k] = 0.
#     else: grapharray[j][k] = 1.
grapharray = grapharray.reshape(1,64,64,1)
graphpreds = model.predict(grapharray, verbose=0)
predict_labels = np.argsort(-graphpreds)
for idx in predict_labels[:]:
  print(numstonames[idx[0]], numstonames[idx[1]], numstonames[idx[2]])




## Check Generator

In [0]:
image_label = "key"
img_h, img_w = 64, 64
num_examples = 100
latent_dim = 100
correct_count = 0
save_index = 0

gen_model = keras.models.load_model("drive/My Drive/output/model/generator/"+image_label+ "_generator.h5")
correct_label = image_label.replace(" ", "_")
print(correct_label)
random_noise = np.random.normal(0, 1, size=[num_examples, latent_dim])
generator_predictions = gen_model.predict(random_noise)
generator_predictions = generator_predictions.reshape(num_examples, img_h, img_w)

plt.figure(figsize=(10, 10))
for i in range(generator_predictions.shape[0]):
    plt.subplot(10, 10, i+1)
    plt.imshow(generator_predictions[i], interpolation='nearest', cmap='gray')
    grapharray = generator_predictions[i]
    #Prepocessing
    for j in range(64):
        for k in range(64):
            if grapharray[j][k] < 0.2 : grapharray[j][k] = 1.
            else: grapharray[j][k] = 0.

    grapharray = np.reshape(grapharray, (1, 64, 64, 1))
    graphpreds = model.predict(grapharray, verbose=0)
    gvs = np.argsort(-graphpreds)
    for idx in gvs[:]:
        print(numstonames[idx[0]], numstonames[idx[1]], numstonames[idx[2]])
        if numstonames[idx[0]] == correct_label : 
          correct_count = correct_count + 1
          save_index = save_index + 1
          # save_images(grapharray, save_index)
    plt.axis('off')
print("Accuracy = ", correct_count / num_examples)
plt.suptitle("Samples")
plt.show()
